# TITLE 
etc.

In [1]:
from pathlib import Path
import pandas as pd
import re

from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates

In [ ]:
# TODO: write docstrings for the functions, eventually comments in the code
# TODO: check code again
# TODO: eventually make more analyses to understand data better -> eventually a pandas profiling (be aware of the number of rows though)
# TODO: make explanations in markdown
# TODO: add weather data and look at it together

In [2]:
data = pd.read_parquet(Path("../data") / "train.parquet")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455163 entries, 48321 to 928462
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 455163 non-null  category      
 1   counter_name               455163 non-null  category      
 2   site_id                    455163 non-null  int64         
 3   site_name                  455163 non-null  category      
 4   bike_count                 455163 non-null  float64       
 5   date                       455163 non-null  datetime64[ns]
 6   counter_installation_date  455163 non-null  datetime64[ns]
 7   counter_technical_id       455163 non-null  category      
 8   latitude                   455163 non-null  float64       
 9   longitude                  455163 non-null  float64       
 10  log_bike_count             455163 non-null  float64       
dtypes: category(4), datetime64[ns](2), float64(4), i

Check the dates in the data set, so that weather data can be merged later correctly.

In [4]:
mask = data["counter_name"] == "Totem 73 boulevard de Sébastopol S-N"
data[mask].date.value_counts()

2020-09-01 03:00:00    1
2021-06-30 06:00:00    1
2021-03-07 05:00:00    1
2021-03-19 23:00:00    1
2021-06-29 07:00:00    1
                      ..
2020-11-02 09:00:00    1
2020-11-24 07:00:00    1
2021-03-05 17:00:00    1
2020-11-02 07:00:00    1
2021-08-09 23:00:00    1
Name: date, Length: 8230, dtype: int64

In [5]:
print(data.date.min())
print(data.date.max())

2020-09-01 01:00:00
2021-08-09 23:00:00


In [6]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [7]:
encoded_dates = _encode_dates(data[["date"]])
encoded_dates.year.unique()

array([2020, 2021])

In [8]:
def _additional_date_variables(X):
    X = X.copy()  # modify a copy of X

    # add seasons
    seasons = {
        1: "winter",
        2: "winter",
        3: "spring",
        4: "spring",
        5: "spring",
        6: "summer",
        7: "summer",
        8: "summer",
        9: "autumn",
        10: "autumn",
        11: "autumn",
        12: "winter",
    }
    X.loc[:, "season"] = X["date"].dt.month.map(seasons)

    public_holidays = []
    school_holidays = {}
    for year in X["date"].dt.year.unique():
        public_holidays.extend(JoursFeries.for_year(year).values())
        school_holidays.update(
            SchoolHolidayDates().holidays_for_year_and_zone(year, "C")
        )

    # add public holidays
    X.loc[:, "public_holiday"] = X["date"].isin(public_holidays)

    # add school holidays names
    school_holidays_name = {
        k: re.sub("\s+|'", "_", re.sub("[éë]", "e", v["nom_vacances"].lower()))
        for k, v in school_holidays.items()
        if v["vacances_zone_c"]
    }
    X.loc[:, "school_holiday_name"] = X["date"].map(school_holidays_name)

    # add school holidays
    school_holidays_bool = [
        k for k, v in school_holidays.items() if v["vacances_zone_c"]
    ]
    X.loc[:, "school_holiday"] = X["date"].isin(school_holidays_bool)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [9]:
additional_dates = _additional_date_variables(data[["date"]])
additional_dates.head()

,season,public_holiday,school_holiday_name,school_holiday
48321,autumn,False,NaN,False
48324,autumn,False,NaN,False
48327,autumn,False,NaN,False
48330,autumn,False,NaN,False
48333,autumn,False,NaN,False


In [10]:
full_data = pd.concat([data, encoded_dates, additional_dates], axis=1)

In [11]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455163 entries, 48321 to 928462
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 455163 non-null  category      
 1   counter_name               455163 non-null  category      
 2   site_id                    455163 non-null  int64         
 3   site_name                  455163 non-null  category      
 4   bike_count                 455163 non-null  float64       
 5   date                       455163 non-null  datetime64[ns]
 6   counter_installation_date  455163 non-null  datetime64[ns]
 7   counter_technical_id       455163 non-null  category      
 8   latitude                   455163 non-null  float64       
 9   longitude                  455163 non-null  float64       
 10  log_bike_count             455163 non-null  float64       
 11  year                       455163 non-null  int6